In [14]:
# Group 16
# Sjoerd Bootsma: 5242053
# Gijs van der Klink: 5389283
# Jelle Weijland: 5093457

#------------------------------------------------------------------------------------------------

import math
import pandas as pd


data_file_path = 'data/AirportData.xlsx'
data_file_path2 = 'data/Group16.xlsx'

#------------------------------------------------------------------------------------------------

def load_airport_data(file_path, sheet_name='Airport'):
    """Laadt de data van vliegvelden uit een Excel-bestand."""
    airport_data = pd.read_excel(file_path, sheet_name=sheet_name, header=None)
    airport_data_transposed = airport_data.transpose()
    airport_data_transposed.columns = airport_data_transposed.iloc[0]
    airport_data_transposed = airport_data_transposed[1:]
    return airport_data_transposed

def calculate_distance(lat1, lon1, lat2, lon2):
    """Bereken de afstand in kilometers tussen twee geografische coördinaten."""
    R = 6371  # Aarde straal in kilometers
    lat1, lon1 = math.radians(lat1), math.radians(lon1)
    lat2, lon2 = math.radians(lat2), math.radians(lon2)
    delta_lat = lat2 - lat1
    delta_lon = lon2 - lon1
    a = math.sin(delta_lat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(delta_lon / 2)**2
    delta_sigma = 2 * math.asin(math.sqrt(a))
    return R * delta_sigma

def calculate_distance_matrix(latitudes, longitudes):
    """Maak een afstandsmatrix voor de opgegeven breedte- en lengtegraden."""
    num_airports = len(latitudes)
    distance_matrix = [[0 for _ in range(num_airports)] for _ in range(num_airports)]
    for i in range(num_airports):
        for j in range(num_airports):
            if i != j:
                distance_matrix[i][j] = calculate_distance(latitudes[i], longitudes[i], latitudes[j], longitudes[j])
    return distance_matrix

def calculate_distance_dataframe(airport_data):
    """Berekent een DataFrame van afstanden tussen vliegvelden."""
    latitudes = airport_data['Latitude (deg)'].astype(float).values
    longitudes = airport_data['Longitude (deg)'].astype(float).values
    city_names = airport_data['IATA code'].tolist()
    distance_matrix = calculate_distance_matrix(latitudes, longitudes)
    distance_df = pd.DataFrame(distance_matrix, index=city_names, columns=city_names)
    return distance_df.round(1)

def save_and_print_fra_distances(distance_df, output_csv_path='DistanceMatrix_FRA.csv'):
    """Filtert de afstanden vanaf Frankfurt (FRA) en slaat ze op in een CSV-bestand."""
    if 'FRA' in distance_df.index and 'FRA' in distance_df.columns:
        fra_distances = distance_df.loc['FRA']
        filtered_distance_df = pd.DataFrame({"Distance to/from FRA": fra_distances})
        filtered_distance_df.to_csv(output_csv_path, index=True)
        print("\nFiltered distance matrix (to/from FRA):")
        print(filtered_distance_df)
    else:
        print("\nFRA not found in the dataset. No filtered distance matrix was saved.")

#------------------------------------------------------------------------------------------------

def process_demand_data(file_path2, sheet_name, output_csv_file):
    # Laad Excel-data en verwijder eerste kolom
    data = pd.read_excel(
        file_path2, 
        sheet_name=sheet_name, 
        header=None, 
        usecols=lambda x: x != 0  
)
    data.columns = [f"Column_{i}" for i in range(1, data.shape[1] + 1)]
    data = data.rename(columns={"Column_1": "Departure", "Column_2": "Arrival"})

    # Label kolommen 3 tot 32 als 0 tot 29
    columns_to_label = [f"Column_{i}" for i in range(3, 33)]  # Kolommen 3 tot 32
    new_labels = list(range(30))  # Labels 0 tot 29
    label_mapping = dict(zip(columns_to_label, new_labels))
    data = data.rename(columns=label_mapping)

    # Filter rijen waar 'FRA' in voorkomt
    fra_related_rows = data[
        (data["Departure"] == "FRA") | (data["Arrival"] == "FRA")
    ].reset_index(drop=True)

    # Sla het resultaat op als CSV-bestand
    fra_related_rows.to_csv(output_csv_file, index=False)



# Laad vliegvelddata
airport_data = load_airport_data(data_file_path)
distance_df = calculate_distance_dataframe(airport_data)
save_and_print_fra_distances(distance_df)


process_demand_data(data_file_path2, 'Group 16', 'demand_data.csv')



Filtered distance matrix (to/from FRA):
     Distance to/from FRA
LHR                 654.8
CDG                 449.0
AMS                 366.6
FRA                   0.0
MAD                1422.1
BCN                1093.6
MUC                 299.1
FCO                 958.1
DUB                1086.9
ARN                1223.0
LIS                1873.8
TXL                 431.7
HEL                1537.9
WAW                 896.2
EDI                1032.9
OTP                1449.0
HER                2114.4
KEF                2401.2
PMO                1378.1
FNC                2836.0


In [15]:
def process_demand_data(input_csv, output_csv):
    """
    Leest de CSV in, voert berekeningen uit voor kolommen 2 en verder en slaat de resultaten op.
    
    Parameters:
    - input_csv: Pad naar het invoer-CSV-bestand.
    - output_csv: Pad waar het uitvoer-CSV-bestand wordt opgeslagen.
    """
    # 1. Lees de CSV in
    df = pd.read_csv(input_csv)

    # Houd non-numerieke kolommen (zoals 'Departure', 'Arrival') apart
    non_numeric_cols = ['Departure', 'Arrival']
    numeric_cols = [c for c in df.columns if c not in non_numeric_cols]

    # 2. Maak een kopie van de originele waarden
    df_orig = df.copy()

    # 3. Voor i >= 2: nieuwe waarde = oude waarde + 0.2*(oude waarde van kolom i-1) + 0.2*(oude waarde van kolom i-2)
    for i in range(2, len(numeric_cols)):
        col_i = numeric_cols[i]
        col_i_m1 = numeric_cols[i-1]  # i-1
        col_i_m2 = numeric_cols[i-2]  # i-2

        # Zorg dat we numeric uitvoeren op de kolommen
        df[col_i] = df_orig[col_i].astype(float) \
                    + 0.2 * df_orig[col_i_m1].astype(float) \
                    + 0.2 * df_orig[col_i_m2].astype(float)

    # 4. Schrijf de nieuwe CSV weg
    df.to_csv(output_csv, index=False)

    # 5. Print om te checken
    print(f"The processed data has been saved to '{output_csv}'")
    print(df)

# Aanroep van de functie
process_demand_data('demand_data.csv', 'new_demand_data.csv')


The processed data has been saved to 'new_demand_data.csv'
   Departure Arrival  0  1    2    3    4              5              6  \
0        LHR     FRA  0  0  0.0  0.0  0.0   54619.615932  105833.134837   
1        CDG     FRA  0  0  0.0  0.0  0.0   21384.845598   83365.185588   
2        AMS     FRA  0  0  0.0  0.0  0.0   47993.707819  153018.009211   
3        FRA     LHR  0  0  0.0  0.0  0.0   62730.384709   68750.824037   
4        FRA     CDG  0  0  0.0  0.0  0.0    9383.982730   11984.424690   
5        FRA     AMS  0  0  0.0  0.0  0.0    8915.280120    3242.234463   
6        FRA     FRA  0  0  0.0  0.0  0.0       0.000000       0.000000   
7        FRA     MAD  0  0  0.0  0.0  0.0  169730.252857   42762.638561   
8        FRA     BCN  0  0  0.0  0.0  0.0   16378.322648   17659.108225   
9        FRA     MUC  0  0  0.0  0.0  0.0   34530.139626   45999.914494   
10       FRA     FCO  0  0  0.0  0.0  0.0   13245.640296   16803.829962   
11       FRA     DUB  0  0  0.0  0.0  0.0

In [16]:
import pandas as pd
import os
import numpy as np

distance_matrix_path = 'DistanceMatrix_FRA.csv'


# Fleet data
fleet_data = pd.DataFrame({
    'Type': ['Small Freighter', 'Mid-size Old Freighter', 'Large Freighter'],
    'Speed': [800, 850, 920],  # Speed in km/h
    'Cargo_Capacity': [23000, 35000, 120000],  # Cargo capacity in kg
    'TAT': [90, 120, 150],  # Average Turn-Around Time in minutes
    'Max_Range': [1500, 3300, 6300],  # Maximum range in km
    'RQ': [1400, 1600, 1800],  # Runway required in meters
    'Lease_Cost': [2143, 4857, 11429],  # Lease cost in EUR/day
    'Fixed_Cost': [750, 1500, 3125],  # Fixed operating cost per flight leg in EUR
    'Time_Cost': [1875, 1938, 3500],  # Cost per hour in EUR
    'Fuel_Cost': [2.5, 5, 9.5],  # Fuel cost parameter
    'Fleet': [2, 2, 1]  # Fleet count
    })


def calculate_operating_costs(distances, fleet_data):
    total_costs_df = pd.DataFrame(index=distances.index)

    for _, row in fleet_data.iterrows():
        type_name = row['Type']
        fixed_cost = row['Fixed_Cost']
        time_cost_param = row['Time_Cost']
        fuel_cost_param = row['Fuel_Cost']
        airspeed = row['Speed']

        # Fixed operating cost (C^k_X)
        fixed_operating_cost = fixed_cost

        # Time-based costs (C^k_Tij)
        time_based_costs = (time_cost_param * distances / airspeed).round(2)

        # Fuel costs (C^k_Fij)
        fuel_costs = (fuel_cost_param * 1.42 *0.97*  distances / 1.5).round(2)

        # Total operating cost (C^k_ij)
        total_costs = fixed_operating_cost + time_based_costs + fuel_costs
        total_costs_df[type_name] = total_costs.round(2)

    return total_costs_df

def calculate_and_save_flight_times(fra_distances, fleet_data, output_csv_path):

    # Maak een DataFrame om vluchtduur op te slaan
    flight_times_df = pd.DataFrame(index=fra_distances.index)

    # Bereken vluchtduur voor elk vliegtuigtype en voeg het toe aan de DataFrame
    for _, row in fleet_data.iterrows():
        type_name = row['Type']
        speed = row['Speed']
        tat = row['TAT']
        
        # Bereken vluchtduur in minuten en rond af naar het dichtstbijzijnde veelvoud van 6
        flight_times_df[type_name] = (
            ((((fra_distances / speed) + 0.5) * 60) + tat)
            .apply(lambda x: int(np.ceil(x / 6.0) * 6))  # Ronden binnen de berekening
        )

    # Sla de vluchtduur op in een CSV-bestand
    flight_times_df.to_csv(output_csv_path, index=True)
    print(f"\nFlight times have been saved to '{output_csv_path}'.")


distances = pd.read_csv(distance_matrix_path, index_col=0)["Distance to/from FRA"]
operating_costs = calculate_operating_costs(distances, fleet_data)
output_csv_path = 'OperatingCosts_FRA.csv'
operating_costs.to_csv(output_csv_path, index=True)


calculate_and_save_flight_times(distances, fleet_data, 'FlightTimes_FRA.csv')


Flight times have been saved to 'FlightTimes_FRA.csv'.


In [17]:
#from here dynamic programming begins
